# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install ..

Found existing installation: todd-ai 0.4.0
Uninstalling todd-ai-0.4.0:
  Successfully uninstalled todd-ai-0.4.0
Note: you may need to restart the kernel to use updated packages.
Processing /Users/bytedance/Developer/todd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for todd-ai: filename=todd_ai-0.4.0-py3-none-any.whl size=105634 sha256=39b0e0179582ac40bd53bee4deecc592718233dfe55597935498ad9d61f4a8f8
  Stored in directory: /private/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/pip-ephem-wheel-cache-y88sdvgq/wheels/15/ef/5a/9fc12e257ce5cef16b333a2ed6c992ff9cbcc9167f7199e6ac
Successfully built todd-ai

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pathlib
import tempfile
from pprint import pprint
from typing import Any, NoReturn, TypedDict, cast

import todd
import torch
import torch.nn.functional as F
import torch.utils.data

Memo = dict[str, Any]

/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
[2023-08-02 11:10:00,664 13384:140704362395200][patches.py:14 todd <module>] INFO: `ipdb` is installed. Using it for debugging.


## Preparation

### Models

In [3]:
@todd.ModelRegistry.register()
class RunnerModel(todd.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.DatasetRegistry.register()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

### Runners

In [7]:
class RunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        y: torch.Tensor = self._strategy.model(batch['x'])
        loss = F.l1_loss(y, batch['y'])
        memo['loss'] = loss
        if 'log' in memo:
            memo['log']['loss'] = f'{loss.item():.3f}'
        return memo

In [8]:
class TrainerMixin(RunnerMixin):

    def _run_iter(self, batch: Batch, memo: Memo) -> Memo:
        memo = super()._run_iter(batch, memo)
        if 'log' in memo:
            model = cast(RunnerModel, self._strategy.module)
            memo['log']['weight'] = f'{model.weight.item():.3f}'
            memo['log']['batch'] = str(batch)
        return memo

In [9]:
@todd.RunnerRegistry.register()
class CustomValidator(RunnerMixin, todd.runners.Validator):
    pass

In [10]:
@todd.RunnerRegistry.register()
class CustomIterBasedTrainer(TrainerMixin, todd.runners.IterBasedTrainer):
    pass

In [11]:
@todd.RunnerRegistry.register()
class CustomEpochBasedTrainer(TrainerMixin, todd.runners.EpochBasedTrainer):
    pass

## Validators

In [12]:
validator_demo = todd.Config(
    type='CustomValidator',
    name='custom_validator',
    dataloader=dict(batch_size=1, dataset=dict(type='RunnerDataset', n=20)),
    strategy=dict(type='BaseStrategy', model=dict(type='RunnerModel')),
)

In [13]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-08-02 11:10:01,319 13384:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp6pd0na5u
└── custom_validator

2 directories, 0 files


In [14]:
validator_demo.callbacks=dict(type='LogCallback', interval=5)

In [15]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        validator_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()
    
    !echo
    !tree $work_dirs

[2023-08-02 11:10:01,707 13384:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:01,711 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.006453 loss=10.000
[2023-08-02 11:10:01,714 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.005764 loss=20.000
[2023-08-02 11:10:01,717 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] ETA 0:00:00.002708 loss=30.000
[2023-08-02 11:10:01,720 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] ETA 0:00:00 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpub6r45sx
└── custom_validator

2 directories, 0 files


## Trainers

In [16]:
trainer_demo = validator_demo.copy()
trainer_demo.pop('type')
trainer_demo.dataloader = todd.Config(
    batch_size=2,
    shuffle=True,
    dataset=dict(type='RunnerDataset', n=67),
)
trainer_demo.optimizer = todd.Config(type='SGD', lr=0.005)


### Iteration Based

In [17]:
iter_based_trainer_demo = trainer_demo.copy()
iter_based_trainer_demo.type = 'CustomIterBasedTrainer'
iter_based_trainer_demo.name = 'custom_iter_based_trainer'
iter_based_trainer_demo.iters = 53

In [18]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        iter_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 11:10:02,013 13384:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,025 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.099830 loss=12.000 weight=0.000 batch={'x': tensor([10,  2]), 'y': tensor([20,  4])}
[2023-08-02 11:10:02,027 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.054189 loss=81.000 weight=0.000 batch={'x': tensor([48, 33]), 'y': tensor([96, 66])}
[2023-08-02 11:10:02,029 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] ETA 0:00:00.037002 loss=118.000 weight=0.000 batch={'x': tensor([55, 63]), 'y': tensor([110, 126])}
[2023-08-02 11:10:02,031 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custo

### Epoch Based

In [19]:
epoch_based_trainer_demo = trainer_demo.copy()
epoch_based_trainer_demo.type = 'CustomEpochBasedTrainer'
epoch_based_trainer_demo.name = 'custom_epoch_based_trainer'
epoch_based_trainer_demo.epochs = 3

In [20]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        epoch_based_trainer_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 11:10:02,058 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,059 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 11:10:02,062 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.056687 loss=119.000 weight=0.000 batch={'x': tensor([62, 57]), 'y': tensor([124, 114])}
[2023-08-02 11:10:02,064 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.045402 loss=80.000 weight=0.000 batch={'x': tensor([29, 51]), 'y': tensor([ 58, 102])}
[2023-08-02 11:10:02,066 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] ETA 0:00:00.039515 loss=9

## Callbacks

### Log

In [21]:
log_callback_demo = validator_demo.copy()
log_callback = log_callback_demo.callbacks
log_callback.with_file_handler = True
log_callback_demo.callbacks = [log_callback]

In [22]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        log_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-08-02 11:10:02,119 13384:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,122 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.005148 loss=10.000
[2023-08-02 11:10:02,123 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.003528 loss=20.000
[2023-08-02 11:10:02,125 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] ETA 0:00:00.001726 loss=30.000
[2023-08-02 11:10:02,127 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] ETA 0:00:00 loss=40.000



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmputthl73d
└── custom_validator
    └── 2023-08-02T11-10-02_119194-08-00.log

2 directories, 1 file

[2023-08-02 11:10:02,119 13384:140704362395200][base.py:53 todd.CustomValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,122 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [5/20] ETA 0:00:00.005148 loss=10.000
[2023-08-02 11:10:02,123 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [10/20] ETA 0:00:00.003528 loss=20.000
[2023-08-02 11:10:02,125 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [15/20] ETA 0:00:00.001726 loss=30.000
[2023-08-02 11:10:02,127 13384:140704362395200][log.py:70 todd.CustomValidator.custom_validator after_run_iter] INFO: Iter [20/20] ETA 0:00:00 loss=40.000


### Optimize

In [23]:
optimize_callback_demo = iter_based_trainer_demo.copy()
optimize_callback = todd.Config(type='OptimizeCallback')
optimize_callback_demo.callbacks = [optimize_callback, log_callback]

In [24]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        optimize_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 11:10:02,672 13384:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,683 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.093504 loss=28.546 weight=0.607 batch={'x': tensor([11, 30]), 'y': tensor([22, 60])}
[2023-08-02 11:10:02,687 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.057074 loss=46.480 weight=1.170 batch={'x': tensor([53, 59]), 'y': tensor([106, 118])}
[2023-08-02 11:10:02,690 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] ETA 0:00:00.042243 loss=6.270 weight=2.110 batch={'x': tensor([50, 64]), 'y': tensor([100, 128])}
[2023-08-02 11:10:02,693 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custo

### Learning Rate Schedule

In [25]:
lr_schedule_callback_demo = iter_based_trainer_demo.copy()
lr_schedule_callback = todd.Config(
    type='LRScheduleCallback',
    lr_scheduler=dict(type='LinearLR', total_iters=10),
)
lr_schedule_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_callback,
    log_callback,
]


In [26]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 11:10:02,770 13384:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,775 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.027667 loss=73.010 weight=0.262 batch={'x': tensor([47, 37]), 'y': tensor([94, 74])} lr=['3.333e-03']
[2023-08-02 11:10:02,778 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.026428 loss=40.358 weight=1.003 batch={'x': tensor([63, 18]), 'y': tensor([126,  36])} lr=['5.000e-03']
[2023-08-02 11:10:02,781 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] ETA 0:00:00.023981 loss=4.562 weight=1.875 batch={'x': tensor([27, 46]), 'y': tensor([54, 92])} lr=['5.000e-03']
[2023-08-02 11:10:02,784 13384:14070436239

In [27]:
lr_schedule_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
lr_schedule_by_epoch_callback = lr_schedule_callback.copy()
lr_schedule_by_epoch_callback.by_epoch = True
lr_schedule_by_epoch_callback_demo.callbacks = [
    optimize_callback,
    lr_schedule_by_epoch_callback,
    log_callback,
]


In [28]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        lr_schedule_by_epoch_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 11:10:02,821 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,822 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 11:10:02,825 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.069782 loss=66.560 weight=0.293 batch={'x': tensor([22, 56]), 'y': tensor([ 44, 112])} lr=['1.667e-03']
[2023-08-02 11:10:02,828 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.060499 loss=29.213 weight=0.575 batch={'x': tensor([10, 31]), 'y': tensor([20, 62])} lr=['1.667e-03']
[2023-08-02 11:10:02,831 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15

### Learning Rate Scaler

In [29]:
lr_scaler_callback_demo = iter_based_trainer_demo.copy()
lr_scaler_callback = todd.Config(
    type='LRScaleCallback',
    lr_scaler=dict(base_batch_size=1),
)
lr_scaler_callback_demo.callbacks = [
    optimize_callback,
    lr_scaler_callback,
    log_callback,
]

In [30]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        lr_scaler_callback_demo,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2023-08-02 11:10:02,903 13384:140704362395200][lr.py:92 todd.CustomIterBasedTrainer.custom_iter_based_trainer _scale_lr] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2023-08-02 11:10:02,904 13384:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,908 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.022032 loss=11.285 weight=1.630 batch={'x': tensor([45, 16]), 'y': tensor([90, 32])}
[2023-08-02 11:10:02,911 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.022708 loss=2.253 weight=2.085 batch={'x': tensor([47,  6]), 'y': tensor([94, 12])}
[2023-08-02 11:10:02,914 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [15/53] ETA 0:00:00.02156

### Checkpoint

In [31]:
checkpoint_callback_demo = iter_based_trainer_demo.copy()
checkpoint_callback = todd.Config(type='CheckpointCallback', interval=10)
checkpoint_callback_demo.callbacks = [checkpoint_callback, log_callback]

In [32]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomIterBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_50 = pathlib.Path(work_dirs) / 'custom_iter_based_trainer' / 'checkpoints' / 'iter_50'
    for f in iter_50.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-08-02 11:10:02,961 13384:140704362395200][base.py:53 todd.CustomIterBasedTrainer.custom_iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:02,993 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [5/53] ETA 0:00:00.011914 loss=19.000 weight=0.000 batch={'x': tensor([ 1, 18]), 'y': tensor([ 2, 36])}
[2023-08-02 11:10:02,997 13384:140704362395200][checkpoint.py:59 todd.CustomIterBasedTrainer.custom_iter_based_trainer _save] INFO: Saving state dict to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_10lxbyi/custom_iter_based_trainer/checkpoints/iter_10
[2023-08-02 11:10:03,000 13384:140704362395200][log.py:70 todd.CustomIterBasedTrainer.custom_iter_based_trainer after_run_iter] INFO: Iter [10/53] ETA 0:00:00.035196 loss=122.000 weight=0.000 batch={'x': tensor([62, 60]), 'y': tensor([124, 120])}
[2023-08-02 11:10:03,003 13384:140704362395200][log.py:70 todd.CustomIterBa

/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_10lxbyi
└── custom_iter_based_trainer
    ├── 2023-08-02T11-10-02_960868-08-00.log
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_20
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_30
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_40
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_50
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├──

In [33]:
checkpoint_by_epoch_callback_demo = epoch_based_trainer_demo.copy()
checkpoint_by_epoch_callback = checkpoint_callback.copy()
checkpoint_by_epoch_callback.update(interval=1, by_epoch=True)
checkpoint_by_epoch_callback_demo.callbacks = [
    checkpoint_by_epoch_callback,
    log_callback,
]

In [34]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_by_epoch_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'custom_epoch_based_trainer' / 'checkpoints' / 'epoch_2'
    for f in epoch_2.glob('*.pth'):
        print(f"{f.name}:")
        pprint(torch.load(f, 'cpu'))
        print()

[2023-08-02 11:10:03,467 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:03,468 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 11:10:03,472 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.081441 loss=94.000 weight=0.000 batch={'x': tensor([31, 63]), 'y': tensor([ 62, 126])}
[2023-08-02 11:10:03,475 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.064198 loss=51.000 weight=0.000 batch={'x': tensor([28, 23]), 'y': tensor([56, 46])}
[2023-08-02 11:10:03,477 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] ETA 0:00:00.054532 loss=105.


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp8t9yrhts
└── custom_epoch_based_trainer
    ├── 2023-08-02T11-10-03_467398-08-00.log
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest
            ├── callbacks.pth
            ├── meta.pth
            ├── model.pth
            ├── optim.pth
            └── strategy.pth

7 directories, 21 files

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                 

In [35]:
checkpoint_load_from_callback_demo = checkpoint_by_epoch_callback_demo.copy()
checkpoint_load_from_callback_demo.callbacks = [
    optimize_callback,
    checkpoint_by_epoch_callback,
    log_callback,
]

In [36]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        checkpoint_load_from_callback_demo, 
        work_dir=dict(root=work_dirs),
        load_from=os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2')
    )
    runner.run()

[2023-08-02 11:10:03,971 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:03,975 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 11:10:03,982 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.128952 loss=38.122 weight=0.505 batch={'x': tensor([ 7, 44]), 'y': tensor([14, 88])}
[2023-08-02 11:10:03,995 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.184423 loss=18.270 weight=1.420 batch={'x': tensor([ 6, 57]), 'y': tensor([ 12, 114])}
[2023-08-02 11:10:04,000 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] ETA 0:00:00.143852 loss=3.00


--------------------



[2023-08-02 11:10:04,521 13384:140704362395200][checkpoint.py:44 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer init] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp7q1kw15s/custom_epoch_based_trainer/checkpoints/epoch_2
[2023-08-02 11:10:04,528 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:04,529 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [3/3]
[2023-08-02 11:10:04,533 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [70/102] ETA 0:00:00.064496 loss=2.870 weight=2.103 batch={'x': tensor([19, 37]), 'y': tensor([38, 74])}
[2023-08-02 11:10:04,536 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [75/102] ETA 0:00:00.028763 loss

### Monitor

In [37]:
class CustomError(RuntimeError):
    pass

In [38]:
class FaultyRunnerMixin(todd.runners.BaseRunner):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError('faulty runner')

In [39]:
@todd.RunnerRegistry.register()
class FaultyValidator(FaultyRunnerMixin, todd.runners.Validator):
    pass

In [40]:
@todd.RunnerRegistry.register()
class FaultyIterBasedTrainer(FaultyRunnerMixin, todd.runners.IterBasedTrainer):
    pass

In [41]:
@todd.RunnerRegistry.register()
class FaultyEpochBasedTrainer(
    FaultyRunnerMixin,
    todd.runners.EpochBasedTrainer,
):
    pass

In [42]:
monitor_callback_demo = validator_demo.copy()
monitor_callback_demo.type = 'FaultyValidator'
monitor_callback = todd.Config(type='MonitorCallback')
monitor_callback_demo.callbacks = [monitor_callback, log_callback]

In [43]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomValidator = todd.RunnerRegistry.build(
        monitor_callback_demo, 
        work_dir=dict(root=work_dirs),
    )
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/custom_validator/*.log

[2023-08-02 11:10:04,609 13384:140704362395200][base.py:53 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:04,611 13384:140704362395200][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x14a486310>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 193, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_13384/2137902126.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner



[2023-08-02 11:10:04,609 13384:140704362395200][base.py:53 todd.FaultyValidator.custom_validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:04,611 13384:140704362395200][monitor.py:28 todd.FaultyValidator.custom_validator __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x14a486310>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 193, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_13384/2137902126.py", line 4, in _run_iter
    raise CustomError('faulty runner')
CustomError: faulty runner


### Priorities

## Strategies

In [44]:
strategy_load_model_from_demo = checkpoint_load_from_callback_demo.copy()

In [45]:
with tempfile.TemporaryDirectory() as work_dirs:
    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    runner: CustomEpochBasedTrainer = todd.RunnerRegistry.build(
        strategy_load_model_from_demo, 
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.load_model_from(os.path.join(work_dirs, 'custom_epoch_based_trainer', 'checkpoints', 'epoch_2', 'model.pth'))
    runner.run()

[2023-08-02 11:10:04,927 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:04,929 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 11:10:04,933 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.083401 loss=10.986 weight=0.707 batch={'x': tensor([ 5, 12]), 'y': tensor([10, 24])}
[2023-08-02 11:10:04,936 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] ETA 0:00:00.071732 loss=14.455 weight=1.510 batch={'x': tensor([53,  6]), 'y': tensor([106,  12])}
[2023-08-02 11:10:04,940 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [15/102] ETA 0:00:00.067744 loss=2.93


--------------------



[2023-08-02 11:10:05,402 13384:140704362395200][base.py:53 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2023-08-02 11:10:05,403 13384:140704362395200][base.py:64 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp9z1rxd0g/custom_epoch_based_trainer/checkpoints/epoch_2/model.pth
[2023-08-02 11:10:05,406 13384:140704362395200][log.py:76 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2023-08-02 11:10:05,411 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [5/102] ETA 0:00:00.084002 loss=7.999 weight=2.203 batch={'x': tensor([50, 29]), 'y': tensor([100,  58])}
[2023-08-02 11:10:05,414 13384:140704362395200][log.py:70 todd.CustomEpochBasedTrainer.custom_epoch_based_trainer after_run_iter] INFO: Iter [10/102] E

## Dry Run

In [46]:
todd.Store.DRY_RUN = True